# PART 2

In [2]:
pip install --upgrade google-cloud-BigQuery

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import io
import random
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
service_path = "/Users/Marcus/Wedge_Final/"
service_file = 'umt-msba-290c4d48a8fb.json'
gbq_proj_id = 'umt-msba'
gbq_dataset_id = 'wedge_transactions'

private_key =service_path + service_file

In [3]:
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [4]:
query_card_nos = """SELECT distinct card_no
    from `umt-msba.wedge_transactions.transArchive*`
    where card_no !=3"""
query_get_card_nos = client.query(query_card_nos,location ='US',)

In [5]:
owners = []

for idx, row in enumerate(query_get_card_nos) :
    card_no = row[0]
    
    owners.append(card_no)

print(len(owners))

27207


In [18]:
random.seed(294)
samp_size = 270
samp_owners = random.choices(owners, k=samp_size)
print(len(samp_owners))

270


In [19]:
owners_join = ",".join([str(owner) for owner in samp_owners])
query = """Select * From `umt-msba.wedge_transactions.transArchive*` Where card_no in (""" + owners_join + ")"
query_own = client.query(query,location ='US',)

In [20]:
headers = ["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"]
with open ("sample_of_owners.txt", "w") as outfile :
    outfile.write ("\t".join(headers) + "\n")
    for row in query_own:
        outfile.write("\t".join([str(item) for item in row]) + "\n")

<_io.TextIOWrapper name='sample_of_owners.txt' mode='w' encoding='UTF-8'>
